In [1]:
from __future__ import annotations

from typing import Union

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
plt.style.use('/home/inca/default.mplstyle')

In [2]:
players_2324_path = "./Quotazioni_Fantacalcio_Stagione_2023_24.csv"
players_2223_path = "./Quotazioni_Fantacalcio_Stagione_2022_23.csv"
stats_2223_path = "./Statistiche_Fantacalcio_Stagione_2022_23.csv"

players_2324_df = pd.read_csv(players_2324_path, delimiter=",", header=1, index_col=False)
players_2223_df = pd.read_csv(players_2223_path, delimiter=",", header=1, index_col=False)
stats_2223_df = pd.read_csv(stats_2223_path, delimiter=",", header=1, index_col=False)

# basic functions

In [3]:
class Filter:

    def __init__(
            self: Filter, column: str, operation: str, value: Union[str, float]) -> None:
        self.column = column
        self.operation = operation
        self.value = value
    
    def apply_to_df(self: Filter, df: pd.DataFrame) -> pd.DataFrame:
        if self.operation == "ge":
            return df[df[self.column] > self.value]
        elif self.operation == "geq":
            return df[df[self.column] >= self.value]
        elif self.operation == "le":
            return df[df[self.column] < self.value]
        elif self.operation == "leq":
            return df[df[self.column] <= self.value]
        elif self.operation == "eq":
            return df[df[self.column] == self.value]
        elif self.operation == "neq":
            return df[df[self.column] != self.value]


def apply_multiple_filters(df: pd.DataFrame, filter_list: list[Filter]) -> pd.DataFrame:
    filtered_df = df.copy()
    for filter in filter_list:
        filtered_df = filter.apply_to_df(filtered_df)
    return filtered_df


def extract_players_mantra_position(df: pd.DataFrame, position: str) -> pd.DataFrame:
    """
    The mantra position is tricky, because they actually include multiple positions with ; as delimiter.
    The easiest way is to select the players where the mantra position column contains also that specific position.
    """
    return df[df["RM"].str.contains(position)]


In [10]:
# example: extract players of a team
Filter("Squadra", "eq", "Salernitana").apply_to_df(players_2324_df)

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
12,6120,P,Por,Ochoa,Salernitana,9,9,0,9,9,0,30,30
26,159,P,Por,Sepe,Salernitana,1,1,0,1,1,0,1,1
40,1947,P,Por,Fiorillo,Salernitana,1,1,0,1,1,0,1,1
43,2338,P,Por,Costil,Salernitana,1,1,0,1,1,0,1,1
76,5481,D,Dd;Ds;E,Mazzocchi,Salernitana,12,12,0,14,14,0,37,41
122,5532,D,Ds;E,Bradaric,Salernitana,7,7,0,7,7,0,11,11
127,660,D,Dc,Gyomber,Salernitana,6,6,0,6,6,0,7,7
153,4897,D,Dc,Pirola,Salernitana,5,5,0,5,5,0,10,10
175,4934,D,Dc,Lovato,Salernitana,4,4,0,4,4,0,6,6
181,5856,D,Dd;E,Sambia,Salernitana,4,4,0,4,4,0,5,5


In [5]:
# example on how to use a filter
filter = Filter("Qt.A M", "geq", 15)
good_players = filter.apply_to_df(players_2324_df)
good_players

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
0,572,P,Por,Meret,Napoli,17,17,0,17,17,0,80,80
1,4312,P,Por,Maignan,Milan,17,17,0,17,17,0,89,89
2,2814,P,Por,Provedel,Lazio,16,16,0,16,16,0,68,68
3,453,P,Por,Szczesny,Juventus,15,15,0,15,15,0,71,71
4,2428,P,Por,Sommer,Inter,15,15,0,15,15,0,67,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,6409,A,Pc,Beltran L.,Fiorentina,16,16,0,16,16,0,82,78
457,1939,A,Pc,Lapadula,Cagliari,15,15,0,15,15,0,41,35
458,2544,A,Pc,Abraham,Roma,15,15,0,15,15,0,58,50
459,2819,A,Pc,Caputo,Empoli,15,15,0,15,15,0,73,60


In [6]:
# example on how to use multiple filters to extract cheap but good goalkeepers
filter_list = [
    Filter("RM", "eq", "Por"),
    Filter("Qt.A M", "geq", 10),
    Filter("FVM M", "leq", 50)
]
filtered_df = apply_multiple_filters(players_2324_df, filter_list)
filtered_df

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
6,5876,P,Por,Di Gregorio,Monza,12,12,0,12,12,0,37,37
7,4431,P,Por,Carnesecchi,Atalanta,11,11,0,11,11,0,28,28
8,133,P,Por,Skorupski,Bologna,10,10,0,10,10,0,32,32
9,2170,P,Por,Milinkovic-Savic V.,Torino,10,10,0,10,10,0,31,31
10,2815,P,Por,Terracciano,Fiorentina,10,10,0,10,10,0,31,31


In [7]:
# example: players who can play also as difensori centrali
difensori_centrali = extract_players_mantra_position(players_2324_df, "Dc")
difensori_centrali

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
65,6066,D,Dd;Dc,Posch,Bologna,16,16,0,18,18,0,53,56
67,4237,D,Dd;Ds;Dc,Danilo,Juventus,15,15,0,18,18,0,44,58
68,4245,D,Dc,Smalling,Roma,15,15,0,15,15,0,55,55
69,4409,D,Dc,Rrahmani,Napoli,14,14,0,14,14,0,44,44
70,6041,D,Dc,Schuurs,Torino,14,14,0,14,14,0,46,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,6244,D,Dc,Matturro,Genoa,1,1,0,1,1,0,1,1
239,6253,D,Dc,N'guessan,Torino,1,1,0,1,1,0,1,1
243,6411,D,Dc,Mateus Lusuardi,Frosinone,1,1,0,1,1,0,1,1
244,6430,D,Dc,Kalaj,Frosinone,1,1,0,1,1,0,1,1


# extracting interesting players: Andrea

In [11]:
atalanta_players = ["Lookman", "Scamacca", "Zapata D.", "Scalvini",
                    "Toure' E.", "Zappacosta", "Kolasinac", "Bakker",
                    "Ederson D.s.", "Koopmeiners", "De Roon", "De Ketelaere"]

bologna_players = ["Zirkzee", "Beukema", "Ferguson", "Moro N.", "Orsolini", "Dominguez"]

cagliari_players = ["Lapadula", "Jankto", "Sulemana I.", "Makoumbou", "Luvumbo", "Nandez"]

empoli_players = ["Baldanzi", "Gyasi", "Caputo", "Henderson", "Cancellieri"]

fiorentina_players = ["Amrabat", "Gonzalez N.", "Arthur Melo", "Milenkovic",
                      "Bonaventura", "Dodo'", "Mina", "Parisi", "Kayode"]

frosinone_players = ["Cuni", "Caso"]

genoa_players = ["Malinovskyi", "Dragusin", "Retegui", "Strootman", "Gudmundsson A.", "Messias", "Aramu"]

inter_players = ["Martinez L.", "Barella", "Dimarco", "Bastoni", "Frattesi"]

juventus_players = ["Danilo", "Weah", "Fagioli", "Miretti", "Kostic", "Locatelli"]

lazio_players = ["Rovella", "Casale", "Provedel", "Lazzari", "Zaccagni", "Luis Alberto", "Kamada", "Immobile"]

lecce_players = ["Rafia", "Maleh", "Baschirotto", "Almqvist", "Strefezza"]

milan_players = ["Chukwueze", "Maignan", "Hernandez T.", "Thiaw", "Kalulu",
                 "Reijnders", "Romero L.", "Krunic", "Rafael Leao", "Bennacer", "Loftus-Cheek"]

monza_players = ["Ciurria", "Di Gregorio", "Colpani", "Pessina", "Mari'", "D'ambrosio"]

napoli_players = ["Rrahmani", "Cajuste", "Zambo Anguissa", "Simeone",
                  "Raspadori", "Di Lorenzo", "Meret", "Mario Rui"]

roma_players = ["Pellegrini Lo.", "Renato Sanches", "Zalewski", "Bove", "Smalling", "N'dicka", "Aouar"]

salernitana_players = ["Mazzocchi", "Candreva", "Ochoa", "Coulibaly L.", "Kastanos"]

sassuolo_players = ["Berardi", "Lauriente'", "Lopez M.", "Matheus Henrique", "Viti", "Mulattieri"]

torino_players = ["Milinkovic-Savic V.", "Vlasic", "Sanabria", "Bellanova",
                  "Schuurs", "Ilic", "Vojvoda", "Ricci S."]

udinese_players = ["Deulofeu", "Beto", "Kamara H."]

verona_players = ["Ngonge", "Folorunsho", "Djuric", "Bonazzoli", "Faraoni"]

interesting_players_andrea = atalanta_players + bologna_players + cagliari_players + empoli_players + fiorentina_players + \
    frosinone_players + genoa_players + inter_players + juventus_players + lazio_players + \
        lecce_players + milan_players + monza_players + napoli_players + roma_players + \
            salernitana_players + sassuolo_players + torino_players + udinese_players + verona_players


In [12]:
andrea_players_2324_df = players_2324_df[np.isin(players_2324_df["Nome"], interesting_players_andrea)]
andrea_players_2223_df = players_2223_df[np.isin(players_2223_df["Nome"], interesting_players_andrea)]
andrea_stats_2223_df = stats_2223_df[np.isin(stats_2223_df["Nome"], interesting_players_andrea)]

In [13]:
print(andrea_players_2324_df.shape)
print(andrea_players_2223_df.shape)
print(andrea_stats_2223_df.shape)

(128, 13)
(93, 13)
(96, 18)


In [14]:
# three players did not finish the season in Serie A last year
# Viti -> Nice
# Malinovskyi -> Marseille
# Scamacca -> West Ham
andrea_stats_2223_df[np.invert(np.isin(andrea_stats_2223_df["Nome"], andrea_players_2223_df["Nome"]))]

,Id,R,Rm,Nome,Squadra,Pv,Mv,Fm,Gf,Gs,Rp,Rc,R+,R-,Ass,Amm,Esp,Au
166,5718,D,Dc,Viti,Empoli,0,0.0,0.00,0,0,0,0,0,0,0,0,0,0
352,4427,C,T,Malinovskyi,Atalanta,12,6.0,6.38,1,0,0,0,0,0,2,2,0,0
556,2137,A,Pc,Scamacca,Sassuolo,0,0.0,0.00,0,0,0,0,0,0,0,0,0,0


In [15]:
# players in Serie A who did not play last year in Serie A
andrea_players_2324_df[np.invert(np.isin(andrea_players_2324_df["Nome"], andrea_stats_2223_df["Nome"]))]

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
75,5067,D,E,Bakker,Atalanta,12,12,0,11,11,0,42,35
95,2640,D,Ds;Dc,Kolasinac,Atalanta,9,9,0,10,10,0,20,23
96,4317,D,Dc,N'dicka,Roma,9,9,0,9,9,0,24,24
108,4210,D,Dc,Mina,Fiorentina,8,8,0,8,8,0,18,18
135,5365,D,Dd;Dc,Dragusin,Genoa,6,6,0,7,7,0,9,10
136,5555,D,Ds;E,Kamara H.,Udinese,6,6,0,6,6,0,10,10
140,6202,D,Dc,Beukema,Bologna,6,6,0,6,6,0,14,14
234,6235,D,Dd;Ds;E,Kayode,Fiorentina,1,1,0,2,2,0,2,3
255,4856,C,W;A,Chukwueze,Milan,19,19,0,18,18,0,97,96
268,4142,C,C;T,Aouar,Roma,15,15,0,15,15,0,47,47


In [16]:
goalkeepers = Filter("R", "eq", "P").apply_to_df(andrea_players_2324_df)
defenders = Filter("R", "eq", "D").apply_to_df(andrea_players_2324_df)
midfielders = Filter("R", "eq", "C").apply_to_df(andrea_players_2324_df)
attackers = Filter("R", "eq", "A").apply_to_df(andrea_players_2324_df)
print(f"Number of goalkeepers: {goalkeepers.shape[0]:d}")
print(f"Number of defenders: {defenders.shape[0]:d}")
print(f"Number of midfielders: {midfielders.shape[0]:d}")
print(f"Number of attackers: {attackers.shape[0]:d}")

Number of goalkeepers: 6
Number of defenders: 34
Number of midfielders: 60
Number of attackers: 28


In [17]:
goalkeepers

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
0,572,P,Por,Meret,Napoli,17,17,0,17,17,0,80,80
1,4312,P,Por,Maignan,Milan,17,17,0,17,17,0,89,89
2,2814,P,Por,Provedel,Lazio,16,16,0,16,16,0,68,68
6,5876,P,Por,Di Gregorio,Monza,12,12,0,12,12,0,37,37
9,2170,P,Por,Milinkovic-Savic V.,Torino,10,10,0,10,10,0,31,31
12,6120,P,Por,Ochoa,Salernitana,9,9,0,9,9,0,30,30


In [18]:
defenders

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
61,254,D,E,Dimarco,Inter,19,19,0,18,18,0,81,72
62,2816,D,Dd;E,Di Lorenzo,Napoli,18,18,0,18,18,0,75,75
63,4292,D,Ds;E,Hernandez T.,Milan,17,17,0,17,17,0,83,86
66,554,D,E,Zappacosta,Atalanta,15,15,0,14,14,0,48,42
67,4237,D,Dd;Ds;Dc,Danilo,Juventus,15,15,0,18,18,0,44,58
68,4245,D,Dc,Smalling,Roma,15,15,0,15,15,0,55,55
69,4409,D,Dc,Rrahmani,Napoli,14,14,0,14,14,0,44,44
70,6041,D,Dc,Schuurs,Torino,14,14,0,14,14,0,46,46
72,2120,D,Dc,Bastoni,Inter,13,13,0,13,13,0,43,43
75,5067,D,E,Bakker,Atalanta,12,12,0,11,11,0,42,35


In [19]:
midfielders

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
247,632,C,W;A,Zaccagni,Lazio,28,28,0,28,28,0,195,195
248,5685,C,C;T,Koopmeiners,Atalanta,24,24,0,24,24,0,114,114
251,2085,C,C;T,Luis Alberto,Lazio,21,21,0,22,22,0,90,94
252,2167,C,W,Orsolini,Bologna,21,21,0,20,20,0,94,94
254,1870,C,C,Barella,Inter,20,20,0,20,20,0,86,86
255,4856,C,W;A,Chukwueze,Milan,19,19,0,18,18,0,97,96
257,4486,C,W,Strefezza,Lecce,18,18,0,17,17,0,76,76
258,5687,C,W;T,Vlasic,Torino,18,18,0,19,19,0,70,78
259,2848,C,C,Frattesi,Inter,17,17,0,17,17,0,82,82
260,4427,C,T,Malinovskyi,Genoa,17,17,0,16,16,0,65,59


In [20]:
attackers

,Id,R,RM,Nome,Squadra,Qt.A,Qt.I,Diff.,Qt.A M,Qt.I M,Diff.M,FVM,FVM M
433,2764,A,Pc,Martinez L.,Inter,37,37,0,37,37,0,395,395
434,785,A,Pc,Immobile,Lazio,32,32,0,32,32,0,332,325
435,4510,A,A,Rafael Leao,Milan,31,31,0,31,31,0,270,275
438,531,A,A,Berardi,Sassuolo,30,30,0,30,30,0,272,277
440,4730,A,A,Lookman,Atalanta,26,26,0,26,26,0,185,187
442,2137,A,Pc,Scamacca,Atalanta,25,25,0,25,25,0,160,152
443,479,A,Pc,Sanabria,Torino,24,24,0,24,24,0,158,148
448,6228,A,Pc,Retegui,Genoa,23,23,0,23,23,0,163,151
449,5694,A,Pc,Beto,Udinese,21,21,0,21,21,0,144,132
450,6060,A,A,Lauriente',Sassuolo,21,21,0,21,21,0,138,147


In [22]:
# example: midfielders with average > 6
Filter("R", "eq", "C").apply_to_df(Filter("Mv", "geq", 6).apply_to_df(andrea_stats_2223_df))

,Id,R,Rm,Nome,Squadra,Pv,Mv,Fm,Gf,Gs,Rp,Rc,R+,R-,Ass,Amm,Esp,Au
316,632,C,W;A,Zaccagni,Lazio,35,6.54,7.44,10,0,0,1,1,0,6,9,0,0
321,2085,C,T,Luis Alberto,Lazio,34,6.34,6.93,6,0,0,2,1,1,7,4,0,0
322,2167,C,W;A,Orsolini,Bologna,32,6.16,7.09,11,0,0,3,3,0,2,6,2,0
323,4486,C,W,Strefezza,Lecce,34,6.31,6.97,8,0,0,3,2,1,4,5,0,0
324,5685,C,C;T,Koopmeiners,Atalanta,33,6.36,7.14,10,0,0,4,2,2,5,7,0,0
325,1870,C,C,Barella,Inter,35,6.26,6.86,6,0,0,0,0,0,6,6,0,0
326,5687,C,W;T,Vlasic,Torino,34,6.22,6.82,5,0,0,0,0,0,6,1,0,0
329,332,C,W,Candreva,Salernitana,35,6.21,6.89,7,0,0,0,0,0,4,1,1,0
330,2848,C,C,Frattesi,Sassuolo,36,6.26,6.79,7,0,0,0,0,0,0,4,0,0
333,5858,C,C;T,Ferguson,Bologna,31,6.24,6.85,7,0,0,0,0,0,0,4,0,0
